Set configuration for this measurement

In [18]:
config_path = r'resonator_PD.toml'


import tomlkit

# Assuming 'config.toml' is your file
with open(config_path, 'r') as file:
    content = file.read()
    resPD_config = tomlkit.parse(content)



Load packages

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime
import tomli_w
import copy

config parser

In [20]:


data_output_folder = resPD_config["output"]["data_path"]


new_config = {
    "hardware":resPD_config["hardware"],
    "sample":resPD_config["sample"],
}
measurement = []

attenuation = resPD_config["hardware"]["attenuation"]
total_time = 0
for measurement_info in resPD_config["resonator"]:
    # print(measurement_info)

    single_resonator = {
        "label": measurement_info["label"],
        "frequency": measurement_info["frequency"],
        "IF_bandwidth":measurement_info["IF_bandwidth"]
    }

    vna_power_setting = measurement_info["power"]
    # print(vna_power_setting)
    if "custom" not in vna_power_setting.keys():
        print("linear spacing power")
        min_power = vna_power_setting["min"] 
        max_power = vna_power_setting["max"] 
        step = vna_power_setting["step"] 
        print(min_power,max_power,step)
        power_range = np.arange( min_power, max_power+step/2, step)
    else:
        power_range = np.array(vna_power_setting["custom"])
        min_power = np.min(power_range)

    power_range = power_range 

    IF_bandwidth = measurement_info["frequency"]["points"]
    bandwith = measurement_info["IF_bandwidth"]

    max_repeat = int(measurement_info["snr"]["max_repeat"])
    min_repeat = int(measurement_info["snr"]["min_repeat"])

    total_repeat_time = 0
    for p in power_range:
        r_sn = measurement_info["snr"]["power_ave_ratio"]
        repeat = 10**((r_sn -(p-attenuation) )/10)
        if repeat < min_repeat: repeat = min_repeat
        if repeat > max_repeat: repeat = max_repeat
        total_repeat_time += repeat
        single_power = copy.deepcopy(single_resonator)
        single_power["power"] = p
        single_power["repeat"] = int(repeat)
        single_power["output"] = f"{data_output_folder}\\{measurement_info['label']}\\att{attenuation}_{str(p)}"

        measurement.append(single_power)
    tot_time = total_repeat_time*(IF_bandwidth/bandwith)
    total_time += tot_time
minute, second = divmod(total_time, 60)
hour, minute = divmod(minute, 60)

print('%02d:%02d:%02d' % (hour, minute, second))

new_config["measurement"] = measurement
# Export to a TOML file
with open(f'power_dep_resonator.toml', 'wb') as file:
    tomli_w.dump(new_config, file)

linear spacing power
-44 -41 1
00:28:33


Create task

Connect with VNA

Measurement

Plot